# Code

In [1]:
its_jupyter_notebook = True

In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import time
import numpy as np
import pickle
import math
import torch
from tqdm.notebook import tqdm
import seaborn as sns
from torch.utils.data import DataLoader
# if its_jupyter_notebook:
#     sys.argv = [''] #Remove this if it's not a jupyter notebook!

import sys
sys.path.insert(0, '..')

from config import *

In [3]:
def create_gene_count(df, unbalanced_ratio):
    gene_count = pd.concat([df[['gene1', 'interacting']].rename({'gene1':'gene'}, axis = 1),
                            df[['gene2','interacting']].rename({'gene2':'gene'}, axis = 1)], axis = 0).reset_index(drop = True)
    g = gene_count.groupby('gene').sum()
    g['not_interacting'] = gene_count.groupby('gene').count() - gene_count.groupby('gene').sum()
    g['not_interacting'] = g['not_interacting']*unbalanced_ratio
    g = g.reset_index() #add the gene column from the index
    g['diff'] = g['not_interacting'] - g['interacting']
    g['abs_diff'] = g['diff'].abs()
    g['tot'] = g['interacting'] + g['not_interacting']
    g['abs_diff_weighted'] = g['tot']*g['abs_diff']
    return g

In [4]:
df_meta = pd.read_csv(os.path.join(metadata_dir, f'training.csv'))
df_genes = pd.read_csv(os.path.join(processed_files_dir, f'df_genes.csv'))
df = pd.read_csv(os.path.join(processed_files_dir, f'df_pairs_full.csv'))
new_cols = df.positive.str.split('_', expand = True)
df['g1'], df['g2'] = new_cols[0],new_cols[1]

new_cols = df.negative.str.split('_', expand = True)
df['g1_neg'], df['g2_neg'] = new_cols[0],new_cols[1]

gene = 'ENSG00000261713'
print(df[(df.g1 == gene)|(df.g2 == gene)].positive.unique())
print(df[(df.g1 == gene)|(df.g2 == gene)].negative.unique())
print(df_meta[((df_meta.gene1 == gene)|(df_meta.gene2 == gene))&(df_meta.interacting == False)].couple.unique())

['ENSG00000122566_ENSG00000261713' 'ENSG00000176049_ENSG00000261713']
['ENSG00000005020_ENSG00000261713' 'ENSG00000112541_ENSG00000261713'
 'ENSG00000122566_ENSG00000204277' 'ENSG00000176049_ENSG00000204277']
['ENSG00000005020_ENSG00000261713' 'ENSG00000112541_ENSG00000261713'
 'ENSG00000196576_ENSG00000261713' 'ENSG00000261713_ENSG00000279905']


In [15]:
class_folder = os.path.join(embedding_dir,'999', '22', 'training', 'class_1')

In [61]:
sample = np.load(os.path.join(class_folder, '217458.npy'))
vec10, vec20 = np.split(sample, 2)

In [62]:
sample = np.load(os.path.join(class_folder, '217459.npy'))
vec11, vec21 = np.split(sample, 2)

In [63]:
sample = np.load(os.path.join(class_folder, '26379.npy'))
vec12, vec22 = np.split(sample, 2)

In [69]:
sample.shape

(5120,)

In [64]:
np.isclose(vec10, vec12).all()

True

In [65]:
np.isclose(vec10, vec11).all()

True

In [66]:
np.isclose(vec12, vec11).all()

True

In [67]:
df_meta[(df_meta.gene1 == 'ENSG00000272195')&(df_meta.interacting)]

,id_sample,couple,gene1,gene2,x1,x2,y1,y2,interacting,policy,...,seed_x2,seed_y1,seed_y2,protein_coding1,protein_coding2,original_length1,original_length2,diff1,diff2,diff
63803,217458,ENSG00000260604_ENSG00000272195,ENSG00000272195,ENSG00000260604,0,1739,203,6192,True,easypos,...,1124,1816,1830,False,False,1739,7060,0.656653,-1.992075,-1.335422
63804,217459,ENSG00000260604_ENSG00000272195,ENSG00000272195,ENSG00000260604,0,1739,58,6047,True,easypos,...,1124,1816,1830,False,False,1739,7060,0.656653,-1.992075,-1.335422
68717,26379,ENSG00000113758_ENSG00000272195,ENSG00000272195,ENSG00000113758,0,1739,0,3072,True,easypos,...,1210,2678,2703,False,True,1739,3072,0.656653,0.786399,1.443052
